# 1. Bibliothèques

In [ ]:
!pip install --upgrade pip
!pip install pandas
#!pip install zstandard
!pip install geopandas
!pip install seaborn
!pip install plotly
!pip install zstandard
!pip install nbformat
!pip install -U kaleido
!pip install scikit-learn==1.5.2
!pip install xgboost
!pip install --upgrade scikit-learn xgboost

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import shapely.geometry as geom
import plotly.express as px

import plotly.graph_objects as go

import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier, XGBRegressor
import xgboost as xgb
from sklearn.metrics import classification_report

In [ ]:
!pip install zstandard

# 2. Charger les données

In [ ]:
#nécessaire si fichiers sur google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root_path = "/content/drive/MyDrive/Master/S3/SDA-E/Projet/données/"
#naiades_path = root_path+"naiades/" # à changer avec le bon chemin

### 1. Station

In [ ]:
df_stations = pd.read_csv("donnees/stations_hb.csv.zst",sep=';',escapechar = '\\')

In [ ]:
# df_stations= pd.read_csv(root_path+"stations_hb.csv.zst",sep=';',escapechar = '\\')

In [ ]:
df_stations.head()
#df_station.columns

### 2. Données physico-chimiques

In [ ]:
f="donnees/donnees_physicochimie.csv.zst"
pc_sample = pd.read_csv(f,nrows=1)
pc_list_cols = pc_sample.columns
pc_list_cat = pc_list_cols[pc_list_cols.str.startswith((
    'Lb','Nom','Mnemo',
    'Cd','Sym','Com'))]

pc_dict_cat = {col: 'category' for col in pc_list_cat}

df_pc = pd.read_csv(
        f,
        sep=',',
        engine='c',
        escapechar='\\',
        dtype=pc_dict_cat,
        parse_dates=[7],
        iterator=False)

df_hb = pd.read_csv("donnees/donnees_hydrobio.csv.zst",sep=',',escapechar = '\\')

In [ ]:
# pc_sample= pd.read_csv(root_path+ "donnees_physicochimie.csv.zst",nrows=1)
# pc_list_cols = pc_sample.columns
# pc_list_cat = pc_list_cols[pc_list_cols.str.startswith((
#     'Lb','Nom','Mnemo',
#     'Cd','Sym','Com'))]
# pc_dict_cat = {col: 'category' for col in pc_list_cat}

In [ ]:
# df_pc = pd.read_csv(root_path+
#         "donnees_physicochimie.csv.zst",
#         sep=',',
#         engine='c',
#         escapechar='\\',
#         dtype=pc_dict_cat,
#         parse_dates=[7],
#         iterator=False)

In [ ]:
df_pc.info()

In [ ]:
df_pc.describe()

In [ ]:
df_pc.columns

In [ ]:
df_pc.shape

# 3. Transformation des données

## Supprimer Date Cohérence I2M2

Nous avons décidé de supprimer les données de 2005 à 2007 pour avoir une cohérence avec les données hydrobiologique car cette plage de temps n'est pas présente dans ses donnéees.


In [ ]:
df_pc['DatePrel'].value_counts().sort_index()
df_pc = df_pc[df_pc['DatePrel'] > '2006-12-31']
df_pc['DatePrel'].value_counts().sort_index()

## Ajout des Lables pour répartir données par saison

Nous avons décidé d'ajouter une colonne Saison et LbSaison. Ce choix se justifie par la volonté de mieux suivre les variations saisonnières et les effets qui prennent du temps à apparaître. Cela nous permettra de comparer des périodes similaires et d'eviter les erreurs en cas d'événements exceptionnels.


In [ ]:
## Regroupe les données par saison
df_pc['Saison'] = df_pc['DatePrel'].dt.month.map({
    12: 1, 1: 1, 2: 1,      # Hiver
    3: 2, 4: 2, 5: 2,       # Printemps
    6: 3, 7: 3, 8: 3,       # Été
    9: 4, 10: 4, 11: 4      # Automne
})

# Ajouter une colonne "LbSaison" pour le libellé de la saison
df_pc['LbSaison'] = df_pc['DatePrel'].dt.month.map({
    12: 'Hiver', 1: 'Hiver', 2: 'Hiver',        # Hiver
    3: 'Printemps', 4: 'Printemps', 5: 'Printemps',  # Printemps
    6: 'Ete', 7: 'Ete', 8: 'Ete',              # Été
    9: 'Automne', 10: 'Automne', 11: 'Automne'  # Automne
})


In [ ]:
# Vérification du résultat
print(df_pc[['DatePrel', 'Saison', 'LbSaison']])

## Visualisation de la quantité de donnée

In [ ]:
# Extraire l'année et le mois de la colonne 'DatePrel'
df_pc['Year'] = pd.to_datetime(df_pc['DatePrel']).dt.year
df_pc['Month'] = pd.to_datetime(df_pc['DatePrel']).dt.month

# Calculer la somme des données pour chaque année
compte_par_annee = df_pc.groupby('Year').size()

# Tracer l'histogramme
plt.figure(figsize=(10, 6))
plt.bar(compte_par_annee.index, compte_par_annee.values, color='skyblue')

# Ajouter des éléments au graphique
plt.title('Quantité totale de données par année', fontsize=14)
plt.xlabel('Année', fontsize=12)
plt.ylabel('Quantité totale de données', fontsize=12)
plt.xticks(compte_par_annee.index, rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()

# Afficher le graphique
plt.show()



## Afficher les stations sur une carte pour séparer en hydroecoregions

Permet de visuellement se representer les stations dans chaque hydroecoregions et de modifier le dataset pour y ajouter le code et le nom de chaque hydroeco

In [ ]:
# Définir la projection Lambert 93
crs_lambert = 'PROJCS["RGF_1993_Lambert_93",GEOGCS["GCS_RGF_1993",DATUM["D_RGF_1993",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",700000.0],PARAMETER["False_Northing",6600000.0],PARAMETER["Central_Meridian",3.0],PARAMETER["Standard_Parallel_1",49.0],PARAMETER["Standard_Parallel_2",44.0],PARAMETER["Latitude_Of_Origin",46.5],UNIT["Meter",1.0]]'

# Colonnes des coordonnées X et Y
x_col = 'CoordXStationMesureEauxSurface'
y_col = 'CoordYStationMesureEauxSurface'


gdf_stations = gpd.GeoDataFrame(
    df_stations,
    crs=crs_lambert,
    geometry=gpd.GeoSeries(df_stations.apply(lambda row: geom.Point(row[x_col], row[y_col]), axis=1))
)

# her = gpd.read_file(root_path+"Hydroecoregion1-shp/Hydroecoregion1.shp")
her = gpd.read_file("./donnees/Hydroecoregions/Hydroecoregion1.shp")

shp_hydroecoregions = her.to_crs(crs_lambert)

station_her = gpd.sjoin(gdf_stations, shp_hydroecoregions, predicate='within').to_crs(crs_lambert)



fig, ax = plt.subplots(figsize=(10, 10))

shp_hydroecoregions.plot(ax=ax, color='grey', edgecolor='black', alpha=0.1, label='Hydroécorégions')

#station_her.plot(ax=ax, marker='o', color='#ca4553', markersize=0.2, label='Stations de mesure')
station_her.plot(column='NomHER1',markersize=1, cmap='twilight', legend=True, ax=ax)

#station_her.head()

# Ajouter une légende et un titre
plt.title("Carte des stations de mesure en France avec hydroécorégions et frontières")
plt.legend()

# Afficher la carte
plt.show()

# 4. Début analyse PC

## Donnée avec parametres selectionnés

Nous avons décider de faire une pré séléection des parametres que nous avons estimé important de garder. Pour estimer les paramètres, nous sous somme sbasés sur les paramètres en lien avec la physico chiique de l'eau et les éléments indicatif du prélèvement et ceux qui potentiellement ont un impact sur la valeur des parametres.

In [ ]:
new_data = df_pc[['CdStationMesureEauxSurface', 'CdSupport', 'DatePrel',
                  'CdParametre', 'RsAna', 'ProfondeurPrel',
                  'LdAna', 'LqAna', 'LsAna', 'LbSaison', 'CdPrelevement']]

new_data['Year'] = pd.to_datetime(new_data['DatePrel']).dt.year
compte_par_annee = new_data.groupby(['Year']).count()

# Calculer le pourcentage pour chaque paramètre par année
compte_par_annee_percent = compte_par_annee.div(compte_par_annee.sum(axis=1), axis=0) * 100


fig = go.Figure()

for param in compte_par_annee_percent.columns:
    fig.add_trace(go.Bar(
        x=compte_par_annee_percent.index,
        y=compte_par_annee_percent[param],
        name=param
    ))

# Mise en forme du graphique
fig.update_layout(
    barmode='stack',
    title="Pourcentage de données par année et paramètre",
    xaxis_title="Année",
    yaxis_title="Pourcentage de données (%)",
    xaxis_tickmode='array',
    xaxis_tickvals=compte_par_annee_percent.index,
    xaxis_tickangle=45
)

# Affichage du graphique
fig.show(renderer='browser')


In [ ]:
new_data = df_pc[['CdStationMesureEauxSurface', 'CdSupport', 'DatePrel',
                   'CdParametre', 'RsAna','ProfondeurPrel',
                   'LdAna', 'LqAna', 'LsAna', 'LbSaison', 'CdPrelevement']]

new_data['Year'] = pd.to_datetime(new_data['DatePrel']).dt.year
compte_par_annee = new_data.groupby('Year').count()

# Tracer un graphique pour chaque paramètre
for param in compte_par_annee.columns:
    plt.figure(figsize=(10, 6))
    plt.bar(compte_par_annee.index, compte_par_annee[param], color='skyblue')
    plt.title(f'Nombre de données par année pour {param}')
    plt.xlabel('Année')
    plt.ylabel('Nombre de données')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Une fois qu'on a vu la répartition des données, j'ai fais un "tri". J'ai retiré les paramètres qui ne sont pas nombreux et mal répartis sur les années. Par la suite, jee regarde le nombre de valeur nul pour chaque colonne que j'ai gardé pour voir l'impact des valeurs manquantes.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Sélectionner les colonnes nécessaires
new_data = df_pc[['CdStationMesureEauxSurface', 'CdSupport', 'DatePrel',
                  'CdParametre', 'LbSaison', 'CdPrelevement', 'RsAna']]

# Extraire l'année de la colonne 'DatePrel'
new_data['Year'] = pd.to_datetime(new_data['DatePrel']).dt.year

# Définir le nombre de colonnes pour le layout des graphiques
ncols = 3  # Nombre de colonnes de subplots (ajusté pour mieux utiliser l'espace)
nrows = (len(new_data.columns) - 1) // ncols + 1  # Calculer le nombre de lignes nécessaires

# Créer la figure et les subplots avec un format paysage
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 7 * nrows))  # Augmenter la largeur pour le format paysage
axes = axes.flatten()  # Aplatir l'array pour accéder facilement aux axes

# Tracer les données pour chaque paramètre
for idx, param in enumerate(new_data.columns[:-1]):
    # Compter les valeurs non nulles et nulles par année pour chaque paramètre
    non_null_count = new_data.groupby('Year')[param].apply(lambda x: x.notnull().sum())
    null_count = new_data.groupby('Year')[param].apply(lambda x: x.isnull().sum())

    # Tracer les données nulles et non nulles
    axes[idx].bar(non_null_count.index - 0.2, non_null_count, width=0.4, label='Non Nulles', color='skyblue')
    axes[idx].bar(null_count.index + 0.2, null_count, width=0.4, label='Nulles', color='red')

    # # Ajouter les annotations sur chaque barre
    # for i, val in enumerate(non_null_count):
    #     axes[idx].text(non_null_count.index[i] - 0.2, val, int(val), ha='center', va='bottom')
    # for i, val in enumerate(null_count):
    #     axes[idx].text(null_count.index[i] + 0.2, val, int(val), ha='center', va='bottom')

    # Ajouter les titres et légendes
    axes[idx].set_title(f'Quantité de données nulles et non nulles par année pour {param}')
    axes[idx].set_xlabel('Année')
    axes[idx].set_ylabel('Nombre de données')
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].legend()

# Ajuster les espacements pour ne pas que les graphiques se chevauchent
plt.tight_layout()

# Afficher le graphique
plt.show()


Valeur nulle insignifiante pour RsAna donc à supprimer

Pour ce qui est des années à garder je pense que de garder les années de 2015 à 2018 est intéressant car celles-ci ont le plus de données à analyser

Nous avons garder les paramètres:
- '**CdStationMesureEauxSurface**': Identifiant de la station de mesure dans le référentiel national Sandre. Ce champ permet d'effectuer une jointure avec les fichiers stations et analyses
- '**CdSupport**': Un support désigne un COMPOSANT DU MILIEU SUR LEQUEL PORTE L’INVESTIGATION, faisant généralement l'objet de prélèvements en vue d'analyses ultérieures, afin d’évaluer sa qualité et celle du milieu. Necessaire pour l'analyse
- '**DatePrel**': La date du début du prélèvement physico-chimique est la date à laquelle commence le prélèvement. La date est donnée au jour pres
- '**CdParametre**': Un paramètre est une propriété du milieu ou d'une partie du milieu qui contribue à en apprécier les caractéristiques et/ou la qualité et/ou l'aptitude à des usages
- '**LbSaison**': Nom de la saison
- '**CdPrelevement**' : La référence du prélèvement physico-chimique et biologique chez le producteur est la référence qu'affecte le producteur de données au prélèvement à des fins de gestion et de correspondance notamment pour la facturation des prestations.
- '**RsAna**':   Le résultat de l'analyse physico-chimique est soit la valeur du résultat du paramètre quantitatif, soit le code de la valeur possible du paramètre qualitatif.

## Nettoyer données

Au vue des graphiques ci-dessus, nous avons pu constater la répartition des données était importante entre 2015 et 2018. Nous nous sommes donc concentrés sur cet intervalle pour faire nos analyses.

Une fois que l'intervalle choisi et les paramètres trouvés, je supprime les valeurs NULLE.


In [ ]:
#Filtrer les données de 2015 à 2018
new_data_filtered = new_data[(new_data['DatePrel'].dt.year >= 2015) & (new_data['DatePrel'].dt.year <= 2018)]
new_data_filtered.shape


#Avant supprimer nombre donnée
print("Avant suppression des valeurs nulles :", new_data_filtered.shape)
# Supprimer les lignes avec des valeurs nulles
new_data_filtered_cleaned = new_data_filtered.dropna()
#Apres supprimer nombre donnée
print("Après suppression des valeurs nulles :", new_data_filtered_cleaned.shape)
new_data_filtered_cleaned.groupby('CdStationMesureEauxSurface').size()

# df = px.data.tips()
# fig = px.box(new_data_filtered_cleaned.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False))
# fig.show()
# station_counts = new_data_filtered_cleaned.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
# station_counts

## Supprimer les doublons

In [ ]:
duplicates = new_data_filtered_cleaned[new_data_filtered_cleaned.duplicated(keep=False)]

# Afficher les doublons
print(f"Nombre de doublons trouvés : {duplicates.shape[0]}")
print(duplicates)


In [ ]:
new_data_filtered_cleaned.drop_duplicates(inplace=True)

In [ ]:
new_data_filtered_cleaned.shape

## Répartiton des saisons sur l'année

In [ ]:
new_data_filtered_cleaned['DatePrel'] = pd.to_datetime(new_data_filtered_cleaned['DatePrel'])
new_data_filtered_cleaned['Year'] = new_data_filtered_cleaned['DatePrel'].dt.year

# Grouper par année et saison, et compter les occurrences
data_by_year_season = new_data_filtered_cleaned.groupby(['Year', 'LbSaison']).size().unstack(fill_value=0)

plt.figure(figsize=(12, 8))
ax = data_by_year_season.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='viridis')

# Ajouter des annotations pour les quantités
for i in range(data_by_year_season.shape[0]):  # Parcourir chaque année (chaque barre)
    cumulative_height = 0  # Hauteur cumulative pour positionner les annotations
    for j, season in enumerate(data_by_year_season.columns):  # Parcourir chaque saison
        value = data_by_year_season.iloc[i, j]
        if value > 0:  # Si la valeur est non nulle
            ax.text(i, cumulative_height + value / 2,  # Position : au centre du segment
                    str(value), ha='center', va='center', fontsize=10, color='white')
            cumulative_height += value  # Mettre à jour la hauteur cumulative


plt.title("Répartition des données par saison pour chaque année", fontsize=16)
plt.xlabel("Année", fontsize=14)
plt.ylabel("Nombre de données", fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.legend(title="Saison", fontsize=12, loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


On peut voir que la quantité de donnée est bien répartis entre les saisons de chaque année.

## Répartition par mois

In [ ]:
# Ajouter des colonnes 'year' et 'month' pour le regroupement
new_data_filtered_cleaned['Year'] = new_data_filtered_cleaned['DatePrel'].dt.year
new_data_filtered_cleaned['Month'] = new_data_filtered_cleaned['DatePrel'].dt.month

# Calculer la quantité de données par mois pour chaque année
monthly_data = new_data_filtered_cleaned.groupby(['Year', 'Month']).size().unstack(fill_value=0)

# Tracer le graphique
monthly_data.plot(kind='bar', stacked=False, figsize=(12, 6))
plt.title("Quantité de données par mois de chaque année")
plt.xlabel("Année - Mois")
plt.ylabel("Nombre de données")
plt.xticks(rotation=45)
plt.legend(title="Mois", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


**A voir si on choisit des mois specifique**

# Verification maintenant des répartition dans les stations

## Répartion sur boxplot des stations pour faire le choix des stations


AVANT ENLEVER VALEUR ABERRANTE

In [ ]:
new_data_filtered_cleaned['Month'] = pd.to_datetime(new_data_filtered_cleaned['DatePrel']).dt.month
new_data_filtered_cleaned['Year'] = pd.to_datetime(new_data_filtered_cleaned['DatePrel']).dt.year

new_data_filtered_cleaned = new_data_filtered_cleaned.dropna(subset=['RsAna'])
preprocess_df = new_data_filtered_cleaned.drop(columns=['CdPrelevement', 'DatePrel','CdSupport','CdParametre', 'LbSaison'])
print(preprocess_df)

df = px.data.tips()
fig = px.box(preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False))
fig.show(renderer='browser')
station_counts = preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
station_counts


APRES AVOIR ENLEVER VALEUR ABERRANTE

In [ ]:
new_data_filtered_cleaned['Month'] = pd.to_datetime(new_data_filtered_cleaned['DatePrel']).dt.month
new_data_filtered_cleaned['Year'] = pd.to_datetime(new_data_filtered_cleaned['DatePrel']).dt.year


new_data_filtered_cleaned = new_data_filtered_cleaned.dropna(subset=['RsAna'])
preprocess_df = new_data_filtered_cleaned.drop(columns=['CdPrelevement', 'DatePrel','CdSupport','CdParametre', 'LbSaison'])
station_counts = preprocess_df.groupby('CdStationMesureEauxSurface').size()

# Étape 5 : Calculer les valeurs aberrantes pour le boxplot
Q1 = station_counts.quantile(0.25)
Q3 = station_counts.quantile(0.75)
IQR = Q3 - Q1

# Définir les bornes pour les valeurs aberrantes
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrer les stations dans la plage sans valeurs aberrantes
stations_filtered = station_counts[(station_counts >= lower_bound) & (station_counts <= upper_bound)].index
preprocess_df_filtered = preprocess_df[preprocess_df['CdStationMesureEauxSurface'].isin(stations_filtered)]

# Afficher le résultat après filtrage
print(preprocess_df_filtered)

# Afficher le boxplot sans valeurs aberrantes
fig = px.box(preprocess_df_filtered.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False))
fig.show(renderer='browser')

station_counts = preprocess_df_filtered.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
station_counts

## Station au dela de 0

In [ ]:
#prs les valeurs composé dana sle boxplot
preprocess_df_filtered = preprocess_df_filtered.groupby('CdStationMesureEauxSurface').filter(lambda g: len(g) > 56 and len(g) <= 462)

station_count_non_null = preprocess_df_filtered.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
station_count_non_null

# station_count_non_null = station_count_non_null[station_count_non_null > 56].sort_values(ascending=False)
# station_count_non_null

In [ ]:
## TO DO
# Supprimer les stations qui sont egale à 0
station_count_non_null

Maintenant faut voir les valeurs mesure dans chaque station pour voir lesquels prendre c'est bcp plus simple pour faire un tri

# Verification des CdParametre

## Répartiton des parametre Physico-chimique

Nous visualisons dans un premier temps, la maniere dont les parametres sont répartis en fonction des saisons pour une premiere approche. Cette visualisation nous permettra d'avoir une idée sur l'impact saisonnier sur les parametres.

## Graphique répartition

In [ ]:
new_data_filtered_cleaned['DatePrel'] = pd.to_datetime(new_data['DatePrel'])
new_data_filtered_cleaned['Year'] = new_data_filtered_cleaned['DatePrel'].dt.year
new_data_filtered_cleaned['Month'] = new_data_filtered_cleaned['DatePrel'].dt.month


years = new_data_filtered_cleaned['Year'].unique()

# Nombre de lignes et de colonnes pour la grille de sous-graphes
n_rows = (len(years) + 1) // 2  # 2 graphiques par ligne
n_cols = 2

# Créer une figure et des sous-graphes
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))
axes = axes.flatten()  # Aplatir le tableau de sous-graphes pour un accès facile

# Boucle à travers chaque année pour créer les graphiques
for idx, year in enumerate(years):
    # Filtrer les données pour l'année courante
    data_year = new_data_filtered_cleaned[new_data_filtered_cleaned['Year'] == year]

    # Compter les occurrences par saison et paramètre
    repartition = data_year.groupby(['LbSaison', 'CdParametre']).size().unstack(fill_value=0)

    # Tracer le graphique dans le sous-graphe correspondant
    ax = axes[idx]
    repartition.plot(kind='bar', stacked=True, ax=ax, colormap='viridis')

    # Ajouter les annotations sur chaque portion du graphique
    for i in range(repartition.shape[0]):  # Pour chaque saison
        cumulative_height = 0  # Initialiser la hauteur cumulative
        for j in range(repartition.shape[1]):  # Pour chaque paramètre
            height = repartition.iloc[i, j]  # Hauteur de la barre courante
            if height > 0:  # Vérifier si la hauteur est positive
                ax.annotate(f'{height}',
                            (i, cumulative_height + height / 2),  # Position ajustée au centre de la barre
                            ha='center', va='center', fontsize=10, color='white')
            cumulative_height += height  # Mettre à jour la hauteur cumulative

    # Titre et labels
    ax.set_title(f'Répartition des données CdParametre en fonction des Saisons pour l\'année {year}')
    ax.set_xlabel('Saison')
    ax.set_ylabel('Nombre d\'occurrences')
    ax.set_xticks(range(repartition.shape[0]))
    ax.set_xticklabels(repartition.index, rotation=0)


plt.tight_layout()
plt.show()



Ce graphique nous permet de sélectionner un paramètre afin d'observer son impact physico-chimique.

### Données par saison

In [ ]:
parametre_count = new_data_filtered_cleaned.groupby('CdParametre').size().sort_values(ascending=True)
parametre_count

In [ ]:
nom=df_pc[['LbLongParamètre','CdParametre']].drop_duplicates().reset_index(drop=True)
nom

Je dois verifier:
- Si les paramètres ont le même impact sur l'eau
- En fonction de la quantité: voir si pertinent de les garder
(Dans mes notes)

Pour les regrouper:
- **Matière organique et oxydable:** 1312, 1313, 1335, 1311, 1841
- **Particules en suspension:** 1295, 1305
- **Matière azotées:** 1319,1339
- **Matière phosporées:** 1350,1433
- **Acidification:** 1302
- **Pesticides sur eau brutes:** 1177
- **Nitrates:** 1340
- **Minéralisation:** 1303
- **Temperature:** 1301



## Quantité de parametre par station

In [ ]:
# Calculer la quantité de CdParametre par station
quantite_par_station = new_data_filtered_cleaned.groupby('CdStationMesureEauxSurface')['CdParametre'].count()

# # Afficher les résultats
# print("Quantité de CdParametre par station :")
# print(quantite_par_station)

print(len(quantite_par_station))

In [ ]:
# Calculer la quantité de CdParametre par station
quantite_par_station = new_data_filtered_cleaned.groupby('CdStationMesureEauxSurface')['CdParametre'].count()

# Filtrer les stations avec une quantité supérieure à 0
stations_non_vides = quantite_par_station[quantite_par_station > 0]

# Afficher les stations correspondantes
print("Stations avec une quantité de CdParametre supérieure à 0 :")
print(stations_non_vides)


In [ ]:
stations_ids = stations_non_vides.index.tolist()
#print("Liste des stations avec des données disponibles :")
#print(stations_ids)

nombre=len(stations_ids)
print(nombre)


In [ ]:
stations_vides = quantite_par_station[quantite_par_station == 0]
dfstations_vides = stations_vides.to_frame()

# Tendance et analyses

Importer les donnees de l'hyrdoecoregion 20 avec la qualite en fonction des Clusters.

## Pour les differents datasets selectionnes

Importation des valeurs et creation des dataset resultants
(HER 9/11/20/...)

In [ ]:
# Pour recuperer uniquement les nitrates 
nitrate_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1340']
oxdissous_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1311']
dbo5_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1313']
phosphore_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1350']

# Dictionnaire des seuils pour chaque HER (uniquement pour le cas general pour le moment)
her_thresholds = {
    2: [0.7078, 0.457, 0.3047, 0.1523],
    7: [0.6916, 0.4362, 0.2908, 0.1454],
    11: [0.7003, 0.5252, 0.3501, 0.1751],
    13: [0.7003, 0.5252, 0.3501, 0.1751],
    15: [0.7003, 0.5164, 0.3443, 0.1721],
    19: [0.7003, 0.5252, 0.3501, 0.1751],
    20: [0.7003, 0.5164, 0.3443, 0.1721],
    9: [0.7003, 0.5164, 0.3443, 0.1721]
}

### Pour le nitrate

In [ ]:
# Dictionnaire pour stocker les DataFrames par valeur de HER
results_by_her = {}

# Parcourir tous les seuils dans her_thresholds
for dftorec in her_thresholds:
    data_nitrate = []  # Stocker les résultats pour les nitrates
    data_oxdissous = []  # Stocker les résultats pour l'oxygène dissous
    data_dbo5 = []  # Stocker les résultats pour l'oxygène dissous
    data_phosphore = []  # Stocker les résultats pour l'oxygène dissous

    # Charger le fichier CSV correspondant
    csv_path = f'./donnees/resultats_releves_H{dftorec}_station_bonne-mauvaise.csv'
    df_hb_resultats_clustering = pd.read_csv(csv_path)

    # Conversion de CdStationMesureEauxSurface en string et harmonisation des formats
    for df in [nitrate_new_data_filtered_cleaned, oxdissous_new_data_filtered_cleaned, dbo5_new_data_filtered_cleaned, phosphore_new_data_filtered_cleaned]:
        df["CdStationMesureEauxSurface"] = df["CdStationMesureEauxSurface"].astype(str)

    df_hb_resultats_clustering["CdStationMesureEauxSurface"] = (
        df_hb_resultats_clustering["CdStationMesureEauxSurface"]
        .astype(str)
        .str.zfill(8)  # Ajouter des zéros devant si nécessaire
    )

    # Obtenir les stations uniques pour le seuil en cours
    stations_pre_dataset = df_hb_resultats_clustering["CdStationMesureEauxSurface"].unique()

    # Filtrer les données pour ces stations
    filtered_df_hb_nitrate = nitrate_new_data_filtered_cleaned[
        nitrate_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
    ]
    filtered_df_hb_oxdissous = oxdissous_new_data_filtered_cleaned[
        oxdissous_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
    ]
    filtered_df_hb_dbo5 = dbo5_new_data_filtered_cleaned[
        dbo5_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
    ]
    filtered_df_hb_phosphore = phosphore_new_data_filtered_cleaned[
        phosphore_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
    ]

    # Fonction pour calculer les médianes et résultats
    def process_data(filtered_data, param_name, data_storage):
        for season in filtered_data['LbSaison'].unique():
            # Filtrer les données pour la saison actuelle
            filtered_season_data = filtered_data[filtered_data['LbSaison'] == season]

            # Assurer que les données sont en format datetime et triées par date
            filtered_season_data["DatePrel"] = pd.to_datetime(filtered_season_data["DatePrel"])
            filtered_season_data = filtered_season_data.sort_values(by=["DatePrel"])

            # Calcul de la médiane pour chaque station
            for station, group in filtered_season_data.groupby("CdStationMesureEauxSurface"):
                # Calcul de la médiane des valeurs RsAna
                median_value = group["RsAna"].median()

                # Récupérer la qualité pour l'affichage (la qualité est toujours la même pour la station)
                tmp = df_hb_resultats_clustering[
                    df_hb_resultats_clustering['CdStationMesureEauxSurface'] == station
                ].iloc[0]

                # Ajouter les résultats sous forme de ligne dans la liste de données
                data_storage.append({
                    "CdStationMesureEauxSurface": station,
                    "Saison": season,
                    f"median{param_name}": median_value,
                    "measureFromHB": tmp['Cluster']
                })

    # Traiter les données pour les nitrates
    process_data(filtered_df_hb_nitrate, "Nitrate", data_nitrate)

    # Traiter les données pour l'oxygène dissous
    process_data(filtered_df_hb_oxdissous, "OxygeneDissous", data_oxdissous)

    # Traiter les données pour l'oxygène dissous
    process_data(filtered_df_hb_dbo5, "DBO5", data_dbo5)

    # Traiter les données pour l'oxygène dissous
    process_data(filtered_df_hb_phosphore, "Phosphore", data_phosphore)

    # Convertir les listes de résultats en DataFrame
    nitrate_results = pd.DataFrame(data_nitrate)
    oxdissous_results = pd.DataFrame(data_oxdissous)
    dbo5_results = pd.DataFrame(data_dbo5)
    phosphore_results = pd.DataFrame(data_phosphore)

    # Fusionner les résultats pour les deux paramètres sur la station et la saison
    merged_results = pd.merge(
        nitrate_results,
        oxdissous_results,
        on=["CdStationMesureEauxSurface", "Saison", "measureFromHB"],
        how="outer"
    )

    merged_results = pd.merge(
        merged_results,
        dbo5_results,
        on=["CdStationMesureEauxSurface", "Saison", "measureFromHB"],
        how="outer"
    )
    merged_results = pd.merge(
        merged_results,
        phosphore_results,
        on=["CdStationMesureEauxSurface", "Saison", "measureFromHB"],
        how="outer"
    )

    # Sauvegarder les résultats pour ce seuil HER
    results_by_her[dftorec] = merged_results

In [ ]:
#Afficher le dernier car j'ai referentiel dessus d'apres le reste du travail effectue
results_by_her[20]

In [ ]:
#Verifications uniquement pour le HER20
# results_by_her[20].isna().sum()
# results_by_her[20].isnull()
# results_by_her[20][results_by_her[20]['medianDBO5'].isnull()]

In [ ]:
# Vérifier les colonnes ayant des valeurs manquantes pour tous les datasets
for i in her_thresholds:
    for column in results_by_her[i].columns:
        if results_by_her[i][column].isnull().any():  # Vérifier si la colonne contient des valeurs manquantes
            median_value = results_by_her[i][column].median()  # Calculer la médiane de la colonne
            results_by_her[i][column].fillna(median_value, inplace=True)  # Remplir les valeurs manquantes par la médiane

In [ ]:
# Mettre sous forme de one hot les valeurs des saisons
for i in her_thresholds:
    results_by_her[i] = pd.get_dummies(results_by_her[i], columns=['Saison'], drop_first=True)

In [ ]:
# De maniere pratique, mettre le resultats a la fin 
for i in her_thresholds:
    # Mettre en derniere position measureFromHB
    columns = [col for col in results_by_her[i].columns if col != 'measureFromHB'] + ['measureFromHB']
    results_by_her[i] = results_by_her[i][columns]

In [ ]:
results_by_her[20]

## Importer les valeurs de Hydro

In [ ]:
df_hb_H20_resultats_clustering = pd.read_csv('./donnees/resultats_releves_H20_station_bonne-mauvaise.csv')
print(df_hb_H20_resultats_clustering)

In [ ]:
# Pour afficher la taille du dataset importe
print(df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'].unique())
print(df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'].unique().shape)

In [ ]:
# Ici le but est de recuperer tous les releves des memes stations que dans le H20
# stations_pre_dataset = df_hb_H20_resultats_clustering["CdStationMesureEauxSurface"].unique()
# filtered_df = nitrate_new_data_filtered_cleaned[
#     nitrate_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
# ]
# print(filtered_df)

In [ ]:
# 1051 releves pour les memes stations que le H20
# filtered_df.shape

## Pour toutes les saisons

### Pour le Nitrate

In [ ]:
# Passage des valeurs de CdStationMesureEauxSurface en string qui est plus simple pour l'analyse (contient des 0 devant)
nitrate_new_data_filtered_cleaned["CdStationMesureEauxSurface"] = nitrate_new_data_filtered_cleaned["CdStationMesureEauxSurface"].astype(str)

df_hb_H20_resultats_clustering["CdStationMesureEauxSurface"] = (
    df_hb_H20_resultats_clustering["CdStationMesureEauxSurface"]
    .astype(str)
    .str.zfill(8)  # Juste parceque les donnees venant de hb n'avait pas le 0 devant
)

stations_pre_dataset = df_hb_H20_resultats_clustering["CdStationMesureEauxSurface"].unique()
# Si des valeurs ont etes oublies
# stations_pre_dataset = ["0" + station if not station.startswith("0") else station for station in stations_pre_dataset]

# Filtrer les lignes dans nitrate_new_data_filtered_cleaned en fonction des valeurs de stations_pre_dataset
filtered_df_H20_hb_nitrate = nitrate_new_data_filtered_cleaned[
    nitrate_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
]

print(filtered_df_H20_hb_nitrate)

In [ ]:
# Pour recuperer uniquement les nitrates 
nitrate_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1340']
nitrate_new_data_filtered_cleaned

In [ ]:
import pandas as pd

# Initialisation de la liste pour stocker les résultats sous forme de données
data = []

# Boucle sur toutes les saisons disponibles
for season in filtered_df_H20_hb_nitrate['LbSaison'].unique():
    # Filtrer les données pour la saison actuelle
    filtered_season_data = filtered_df_H20_hb_nitrate[filtered_df_H20_hb_nitrate['LbSaison'] == season]

    # Assurer que les données sont en format datetime et triées par date
    filtered_season_data["DatePrel"] = pd.to_datetime(filtered_season_data["DatePrel"])
    filtered_season_data = filtered_season_data.sort_values(by=["DatePrel"])

    # Calcul de la médiane pour chaque station
    for station, group in filtered_season_data.groupby("CdStationMesureEauxSurface"):
        # Calcul de la médiane des valeurs RsAna
        median_value = group["RsAna"].median()

        # Récupérer la qualité pour l'affichage (la qualité est toujours la même pour la station)
        tmp = df_hb_H20_resultats_clustering[df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'] == station].iloc[0]

        # Ajouter les résultats sous forme de ligne dans la liste de données
        data.append({
            "CdStationMesureEauxSurface": station,
            "Saison": season,
            "medianNitrate": median_value,
            "measureFromHB": tmp['Qualite']
        })

        # Visualiser la série temporelle avec la médiane
        # plt.figure()
        # plt.plot(group["DatePrel"], group["RsAna"], label="Valeurs mesurées")
        # plt.axhline(y=median_value, color="red", linestyle="--", label=f"Médiane: {median_value:.2f}")
        # plt.title(f"Série temporelle pour la station {station} - Saison {season} (Qualité: {tmp['Qualite']})")
        # plt.xlabel("Date")
        # plt.ylabel("RsAna")
        # plt.legend()
        # plt.show()


In [ ]:
# Conversion de la liste de dictionnaires en DataFrame
df_results_median_class_nitrate = pd.DataFrame(data)

# Créer un DataFrame pivoté avec la saison comme colonnes et station comme index
df_result_med_nitrate = df_results_median_class_nitrate.pivot_table(
    index="CdStationMesureEauxSurface",
    columns="Saison",
    values=["medianNitrate"],
    aggfunc="first"
)

# Extraire la qualité pour chaque station (elle est la même pour toutes les saisons)
# df_quality = df_results_median_class_nitrate.drop_duplicates(subset="CdStationMesureEauxSurface")[["CdStationMesureEauxSurface", "measureFromHB"]]

# Joindre la qualité au DataFrame pivoté
# df_result_med_nitrate["measureFromHB"] = df_quality.set_index("CdStationMesureEauxSurface")["measureFromHB"]


In [ ]:
df_result_med_nitrate

### Pour oxygène dissous 

In [ ]:
# Pour recuperer uniquement les nitrates 
oxdissous_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1311']
oxdissous_new_data_filtered_cleaned

In [ ]:
# Passage des valeurs de CdStationMesureEauxSurface en string qui est plus simple pour l'analyse (contient des 0 devant)
oxdissous_new_data_filtered_cleaned["CdStationMesureEauxSurface"] = oxdissous_new_data_filtered_cleaned["CdStationMesureEauxSurface"].astype(str)

stations_pre_dataset = df_hb_H20_resultats_clustering["CdStationMesureEauxSurface"].unique()

# Filtrer les lignes dans nitrate_new_data_filtered_cleaned en fonction des valeurs de stations_pre_dataset
filtered_df_H20_hb_oxdissous = oxdissous_new_data_filtered_cleaned[
    oxdissous_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
]

filtered_df_H20_hb_oxdissous

In [ ]:
# Initialisation de la liste pour stocker les résultats sous forme de données
data = []

# Boucle sur toutes les saisons disponibles
for season in filtered_df_H20_hb_oxdissous['LbSaison'].unique():
    # Filtrer les données pour la saison actuelle
    filtered_season_data = filtered_df_H20_hb_oxdissous[filtered_df_H20_hb_oxdissous['LbSaison'] == season]

    # Assurer que les données sont en format datetime et triées par date
    filtered_season_data["DatePrel"] = pd.to_datetime(filtered_season_data["DatePrel"])
    filtered_season_data = filtered_season_data.sort_values(by=["DatePrel"])

    # Calcul de la médiane pour chaque station
    for station, group in filtered_season_data.groupby("CdStationMesureEauxSurface"):
        # Calcul de la médiane des valeurs RsAna
        median_value = group["RsAna"].median()

        # Récupérer la qualité pour l'affichage (la qualité est toujours la même pour la station)
        tmp = df_hb_H20_resultats_clustering[df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'] == station].iloc[0]

        # Ajouter les résultats sous forme de ligne dans la liste de données
        data.append({
            "CdStationMesureEauxSurface": station,
            "Saison": season,
            "medianOxDissous": median_value,
            "measureFromHB": tmp['Qualite']
        })

        # Visualiser la série temporelle avec la médiane
        # plt.figure()
        # plt.plot(group["DatePrel"], group["RsAna"], label="Valeurs mesurées")
        # plt.axhline(y=median_value, color="red", linestyle="--", label=f"Médiane: {median_value:.2f}")
        # plt.title(f"Série temporelle pour la station {station} - Saison {season} (Qualité: {tmp['Qualite']})")
        # plt.xlabel("Date")
        # plt.ylabel("RsAna")
        # plt.legend()
        # plt.show()

In [ ]:
# Conversion de la liste de dictionnaires en DataFrame
df_results_median_class_oxdissous = pd.DataFrame(data)

# Créer un DataFrame pivoté avec la saison comme colonnes et station comme index
df_result_med_oxdissous = df_results_median_class_oxdissous.pivot_table(
    index="CdStationMesureEauxSurface",
    columns="Saison",
    values=["medianOxDissous"],
    aggfunc="first"
)

# Extraire la qualité pour chaque station (elle est la même pour toutes les saisons)
# df_quality = df_results_median_class_oxdissous.drop_duplicates(subset="CdStationMesureEauxSurface")[["CdStationMesureEauxSurface", "measureFromHB"]]

# Joindre la qualité au DataFrame pivoté
# df_result_med_oxdissous["measureFromHB"] = df_quality.set_index("CdStationMesureEauxSurface")["measureFromHB"]


In [ ]:
df_result_med_oxdissous

### Pour DBO5

In [ ]:
# Pour recuperer uniquement les nitrates 
dbo5_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1313']
dbo5_new_data_filtered_cleaned

In [ ]:
# Passage des valeurs de CdStationMesureEauxSurface en string qui est plus simple pour l'analyse (contient des 0 devant)
dbo5_new_data_filtered_cleaned["CdStationMesureEauxSurface"] = dbo5_new_data_filtered_cleaned["CdStationMesureEauxSurface"].astype(str)

stations_pre_dataset = df_hb_H20_resultats_clustering["CdStationMesureEauxSurface"].unique()

# Filtrer les lignes dans nitrate_new_data_filtered_cleaned en fonction des valeurs de stations_pre_dataset
filtered_df_H20_hb_dbo5 = dbo5_new_data_filtered_cleaned[
    dbo5_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
]

filtered_df_H20_hb_dbo5

In [ ]:
# Initialisation de la liste pour stocker les résultats sous forme de données
data = []

# Boucle sur toutes les saisons disponibles
for season in filtered_df_H20_hb_dbo5['LbSaison'].unique():
    # Filtrer les données pour la saison actuelle
    filtered_season_data = filtered_df_H20_hb_dbo5[filtered_df_H20_hb_dbo5['LbSaison'] == season]

    # Assurer que les données sont en format datetime et triées par date
    filtered_season_data["DatePrel"] = pd.to_datetime(filtered_season_data["DatePrel"])
    filtered_season_data = filtered_season_data.sort_values(by=["DatePrel"])

    # Calcul de la médiane pour chaque station
    for station, group in filtered_season_data.groupby("CdStationMesureEauxSurface"):
        # Calcul de la médiane des valeurs RsAna
        median_value = group["RsAna"].median()

        # Récupérer la qualité pour l'affichage (la qualité est toujours la même pour la station)
        tmp = df_hb_H20_resultats_clustering[df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'] == station].iloc[0]

        # Ajouter les résultats sous forme de ligne dans la liste de données
        data.append({
            "CdStationMesureEauxSurface": station,
            "Saison": season,
            "medianDBO5": median_value,
            "measureFromHB": tmp['Qualite']
        })

        # Visualiser la série temporelle avec la médiane
        # plt.figure()
        # plt.plot(group["DatePrel"], group["RsAna"], label="Valeurs mesurées")
        # plt.axhline(y=median_value, color="red", linestyle="--", label=f"Médiane: {median_value:.2f}")
        # plt.title(f"Série temporelle pour la station {station} - Saison {season} (Qualité: {tmp['Qualite']})")
        # plt.xlabel("Date")
        # plt.ylabel("RsAna")
        # plt.legend()
        # plt.show()

In [ ]:
# Conversion de la liste de dictionnaires en DataFrame
df_results_median_class_dbo5 = pd.DataFrame(data)

# Créer un DataFrame pivoté avec la saison comme colonnes et station comme index
df_result_med_dbo5 = df_results_median_class_dbo5.pivot_table(
    index="CdStationMesureEauxSurface",
    columns="Saison",
    values=["medianDBO5"],
    aggfunc="first"
)

# Extraire la qualité pour chaque station (elle est la même pour toutes les saisons)
# df_quality = df_results_median_class_oxdissous.drop_duplicates(subset="CdStationMesureEauxSurface")[["CdStationMesureEauxSurface", "measureFromHB"]]

# Joindre la qualité au DataFrame pivoté
# df_result_med_dbo5["measureFromHB"] = df_quality.set_index("CdStationMesureEauxSurface")["measureFromHB"]


In [ ]:
df_result_med_dbo5

### Pour le phosphore

In [ ]:
# Pour recuperer uniquement les nitrates 
phosphore_new_data_filtered_cleaned = new_data_filtered_cleaned[new_data_filtered_cleaned['CdParametre'] == '1350']
phosphore_new_data_filtered_cleaned

In [ ]:
# Passage des valeurs de CdStationMesureEauxSurface en string qui est plus simple pour l'analyse (contient des 0 devant)
phosphore_new_data_filtered_cleaned["CdStationMesureEauxSurface"] = phosphore_new_data_filtered_cleaned["CdStationMesureEauxSurface"].astype(str)

stations_pre_dataset = df_hb_H20_resultats_clustering["CdStationMesureEauxSurface"].unique()

# Filtrer les lignes dans nitrate_new_data_filtered_cleaned en fonction des valeurs de stations_pre_dataset
filtered_df_H20_hb_phosphore = phosphore_new_data_filtered_cleaned[
    phosphore_new_data_filtered_cleaned["CdStationMesureEauxSurface"].isin(stations_pre_dataset)
]

filtered_df_H20_hb_phosphore

In [ ]:
# Initialisation de la liste pour stocker les résultats sous forme de données
data = []

# Boucle sur toutes les saisons disponibles
for season in filtered_df_H20_hb_phosphore['LbSaison'].unique():
    # Filtrer les données pour la saison actuelle
    filtered_season_data = filtered_df_H20_hb_phosphore[filtered_df_H20_hb_phosphore['LbSaison'] == season]

    # Assurer que les données sont en format datetime et triées par date
    filtered_season_data["DatePrel"] = pd.to_datetime(filtered_season_data["DatePrel"])
    filtered_season_data = filtered_season_data.sort_values(by=["DatePrel"])

    # Calcul de la médiane pour chaque station
    for station, group in filtered_season_data.groupby("CdStationMesureEauxSurface"):
        # Calcul de la médiane des valeurs RsAna
        median_value = group["RsAna"].median()

        # Récupérer la qualité pour l'affichage (la qualité est toujours la même pour la station)
        tmp = df_hb_H20_resultats_clustering[df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'] == station].iloc[0]

        # Ajouter les résultats sous forme de ligne dans la liste de données
        data.append({
            "CdStationMesureEauxSurface": station,
            "Saison": season,
            "medianPhosphore": median_value,
            "measureFromHB": tmp['Cluster']
        })

        # Visualiser la série temporelle avec la médiane
        # plt.figure()
        # plt.plot(group["DatePrel"], group["RsAna"], label="Valeurs mesurées")
        # plt.axhline(y=median_value, color="red", linestyle="--", label=f"Médiane: {median_value:.2f}")
        # plt.title(f"Série temporelle pour la station {station} - Saison {season} (Qualité: {tmp['Qualite']})")
        # plt.xlabel("Date")
        # plt.ylabel("RsAna")
        # plt.legend()
        # plt.show()

In [ ]:
# Conversion de la liste de dictionnaires en DataFrame
df_results_median_class_phosphore = pd.DataFrame(data)

# Créer un DataFrame pivoté avec la saison comme colonnes et station comme index
df_result_med_phosphore = df_results_median_class_phosphore.pivot_table(
    index="CdStationMesureEauxSurface",
    columns="Saison",
    values=["medianPhosphore"],
    aggfunc="first"
)

# Extraire la qualité pour chaque station (elle est la même pour toutes les saisons)
df_quality = df_results_median_class_phosphore.drop_duplicates(subset="CdStationMesureEauxSurface")[["CdStationMesureEauxSurface", "measureFromHB"]]

# Joindre la qualité au DataFrame pivoté
df_result_med_phosphore["measureFromHB"] = df_quality.set_index("CdStationMesureEauxSurface")["measureFromHB"]



In [ ]:
df_result_med_phosphore

## Concatener tous les datasets

In [ ]:
results_concats = pd.concat([df_result_med_oxdissous, df_result_med_nitrate, df_result_med_dbo5, df_result_med_phosphore], axis=1)

In [ ]:
results_concats

In [ ]:
results_concats.columns = ['_'.join(col).strip() for col in results_concats.columns.values]
results_concats.rename(columns={'measureFromHB_': 'measureFromHB'}, inplace=True)

In [ ]:
results_concats = results_concats.dropna(subset=['measureFromHB'])

In [ ]:
columns_to_replace = results_concats.columns.difference(['measureFromHB'])
# Remplacement des NaN par la moyenne
results_concats[columns_to_replace] = results_concats[columns_to_replace].apply(
    lambda col: col.fillna(col.mean())
)

In [ ]:
results_concats

### XGBoost

Pour determiner si on peut trouver des patterns dans les donnees, il faut utiliser un algorithme tel que XGboost.

In [ ]:
# Separer le dataset en X et y pour travailler sur chaque partie
X = results_concats.drop(['measureFromHB'], axis=1)
y = results_concats['measureFromHB']

In [ ]:
# Train test split tres petit 80% train 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_train.shape

### XGBoost sur toutes les HER

In [ ]:
for i in her_thresholds:
    # Vérification et nettoyage des données
    if 'CdStationMesureEauxSurface' in results_by_her[i].columns:
        results_by_her[i].drop(columns='CdStationMesureEauxSurface', inplace=True)

    # Définir X (features) et y (target)
    X = results_by_her[i].drop(['measureFromHB'], axis=1)
    y = results_by_her[i]['measureFromHB']

    # Séparer les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Area Under the Curve (AUC) pour évaluer les performances d'un modèle en termes de séparation des classes.
    # logloss : Logarithmic Loss (par défaut pour la classification binaire).
    
    # Instancier et entraîner le modèle XGBoost
    model = XGBClassifier(
        eval_metric='auc',  # Métrique de validation par défaut
        use_label_encoder=False,  # Important pour éviter un warning avec les versions récentes
        random_state=42          # Assurer la reproductibilité
    )
    model.fit(X_train, y_train)

    # Prédictions et évaluation
    y_pred = model.predict(X_test)
    print(f'Resultats pour le HER {i}')
    print(classification_report(y_test, y_pred))

    # Importance des caractéristiques
    importance = model.get_booster().get_score(importance_type='weight')  # 'gain', 'cover', ou 'weight'
    print("Importance des caractéristiques :", importance)

    # Visualisation de l'importance des caractéristiques
    xgb.plot_importance(model, importance_type='weight')  # Peut être changé en 'gain' ou 'cover'
    plt.title(f'Importance des caractéristiques pour le HER {i}')
    plt.show()


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy as np

# Paramètres pour RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 1, 10],
    'reg_lambda': [1, 10, 50],
}

# Boucle pour chaque HER
for i in her_thresholds:
    print(f'--- Traitement pour le HER {i} ---')

    # Vérifier la colonne spécifique et la retirer si elle existe
    if 'CdStationMesureEauxSurface' in results_by_her[i].columns:
        results_by_her[i].drop(columns='CdStationMesureEauxSurface', inplace=True)

    X = results_by_her[i].drop(['measureFromHB'], axis=1)
    y = results_by_her[i]['measureFromHB']

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Modèle XGBoost
    model = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False)

    # Configuration de RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=50,  # Nombre d'itérations aléatoires
        scoring='f1_macro',  # Métrique utilisée
        cv=3,  # Validation croisée à 3 plis
        random_state=42,
        verbose=1
    )

    # Entraîner le modèle avec RandomizedSearch
    random_search.fit(X_train, y_train)

    # Meilleurs paramètres
    print("Meilleurs paramètres :", random_search.best_params_)
    print("Meilleur score (validation croisée) :", random_search.best_score_)

    # Évaluer sur les données de test
    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test)

    print(f'Classification Report pour le HER {i}:')
    print(classification_report(y_test, y_pred))

    # Importance des caractéristiques
    importance = best_model.get_booster().get_score(importance_type='weight')
    print(f'Importance des caractéristiques pour le HER {i} : {importance}')

    # Visualiser l'importance des caractéristiques
    xgb.plot_importance(best_model, importance_type='weight')
    plt.show()


In [ ]:
if 'medianOxygeneDissous' in results_by_her[20].columns:
    print(True)

In [ ]:
X_train.shape

## Uniquement pour ete (Ne pas faire tourner, precedent avec toutes les saisons)

In [ ]:
# Pour simplifier on ne prend que l'ete pour le moment
# filtered_df_H20_hb_nitrate_Ete = filtered_df_H20_hb_nitrate[filtered_df_H20_hb_nitrate['LbSaison'] == 'Ete']

In [ ]:
# Mettre le dataset en date time pour etre sur que c'est bien dans l'ordre chronologique (n'a finalement pas d'utilite)
# filtered_df_H20_hb_nitrate_Ete["DatePrel"] = pd.to_datetime(filtered_df_H20_hb_nitrate_Ete["DatePrel"])
# filtered_df_H20_hb_nitrate_Ete = filtered_df_H20_hb_nitrate_Ete.sort_values(by=["DatePrel"])

In [ ]:
# filtered_df_H20_hb_nitrate

In [ ]:
# filtered_df_H20_hb_nitrate_Ete

In [ ]:
# Tres interessant mais la regression n'a certainement pas d'interet. La mediane semble etre plus utile.

# from sklearn.linear_model import LinearRegression
# # Calcul de la tendance pour chaque station
# results = {}
# for station, group in filtered_df_H20_hb_nitrate_Ete.groupby("CdStationMesureEauxSurface"):
#     # Convertir les dates en valeurs numériques pour la régression
#     group["DateNum"] = (group["DatePrel"] - group["DatePrel"].min()).dt.days
    
#     # Modèle de régression linéaire
#     X = group["DateNum"].values.reshape(-1, 1)
#     y = group["RsAna"].values
#     model = LinearRegression().fit(X, y)
    
#     # Stocker les résultats
#     results[station] = {
#         "slope": model.coef_[0],
#         "intercept": model.intercept_,
#         "trend_line": model.predict(X),
#     }
#     tmp = df_hb_H20_resultats_clustering[df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'] == station].iloc[0]
    
#     # Visualiser la série temporelle
#     plt.figure()
#     plt.plot(group["DatePrel"], group["RsAna"], label="Valeurs mesurées")
#     plt.plot(group["DatePrel"], results[station]["trend_line"], label="Tendance", linestyle="--")
#     plt.title(f"Série temporelle pour la station {station} (de base dit {tmp['Qualite']})")
#     plt.xlabel("Date")
#     plt.ylabel("RsAna")
#     plt.legend()
#     plt.show()

In [ ]:
# Calcul de la médiane pour chaque station
# results_median_class_nitrate = {}
# for station, group in filtered_df_H20_hb_nitrate_Ete.groupby("CdStationMesureEauxSurface"):
#     # Calcul de la médiane des valeurs RsAna
#     median_value = group["RsAna"].median()

#     # Récupérer la qualité pour l'affichage
#     tmp = df_hb_H20_resultats_clustering[df_hb_H20_resultats_clustering['CdStationMesureEauxSurface'] == station].iloc[0]
    
#     # Stocker les résultats
#     results_median_class_nitrate[station] = {
#         "medianNitrate": median_value,
#         "measureFromHB": tmp['Qualite']
#     }
    
#     # Visualiser la série temporelle avec la médiane
#     plt.figure()
#     plt.plot(group["DatePrel"], group["RsAna"], label="Valeurs mesurées")
#     plt.axhline(y=median_value, color="red", linestyle="--", label=f"Médiane: {median_value:.2f}")
#     plt.title(f"Série temporelle pour la station {station} (de base dit {tmp['Qualite']})")
#     plt.xlabel("Date")
#     plt.ylabel("RsAna")
#     plt.legend()
#     plt.show()


In [ ]:
# results_median_class_nitrate

In [ ]:
# Pour ajouter si bon ou mauvais
# Extraire la qualité pour chaque station (elle est la même pour toutes les saisons)
# df_quality = df_results_median_class_nitrate.drop_duplicates(subset="CdStationMesureEauxSurface")[["CdStationMesureEauxSurface", "measureFromHB"]]

# Joindre la qualité au DataFrame pivoté
# df_result_med_nitrate["measureFromHB"] = df_quality.set_index("CdStationMesureEauxSurface")["measureFromHB"]